In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy
import spacy
import pathlib
from pathlib import Path

from funnel_transformer import *
from data_loader import *



In [13]:
from data_loader import *
import torch
from torch.utils.data import Dataset, DataLoader

data = EnFrDataset(used_abridged_data=True, max_seq_length=100)

# print(data.__getitem__(0))
# print(data.__len__())

train_dataloader = DataLoader(data, batch_size=32, shuffle=False, num_workers=0)
train_seqence_pair = next(iter(train_dataloader))
print(f"Feature batch shape: {train_seqence_pair.size()}")
print(train_seqence_pair[0])
print(train_seqence_pair[0,0])


Reading the dataframe and storing untokenized pairs...


100%|██████████| 5000/5000 [00:00<00:00, 248633.86it/s]


Adding sentences to Langs amd geting data pairs...


100%|██████████| 5000/5000 [00:00<00:00, 8025.17it/s] 


Creating tokenized pairs of english and french sentences...


100%|██████████| 5000/5000 [00:00<00:00, 17749.90it/s]

Feature batch shape: torch.Size([32, 2, 100])
tensor([[ 0., 34., 35., 36., 34., 37., 36., 38., 39., 40., 36., 41., 42., 43.,
         44., 36., 45., 36., 46., 36., 47., 36., 48., 36., 49., 36., 50., 51.,
         36., 52., 36., 53., 54., 55., 56., 57., 44., 58.,  1.,  2.,  2.,  2.,
          2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
          2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
          2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
          2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
          2.,  2.],
        [ 0., 34., 35., 36., 37., 38., 39., 34., 35., 36., 40., 41., 39., 42.,
         43., 39., 40., 44., 45., 46., 47., 48., 49., 39., 50., 39., 51., 39.,
         52., 39., 53., 39., 54., 39., 55., 56., 39., 57., 39., 58., 59., 60.,
         47., 61.,  1.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
          2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  

In [68]:
from data_loader import *
from funnel_transformer import *
from torch.utils.data import DataLoader
# full_dataset_path = Path('data/en-fr.csv')
# abridged_dataset_path = Path('data/en-fr-abridged.csv')

# ds = pd.read_csv(abridged_dataset_path)
# english, french, pairs = read_lang('en', 'fr', abridged_dataset)

# src_vocab_size = 5000
# tgt_vocab_size = 5000
# d_model = 512
# num_heads = 8
# num_layers = 6
# d_ff = 2048
# max_seq_length = 100
# dropout = 0.1

data = EnFrDataset(used_abridged_data=True, max_seq_length=100)
train_dataloader = DataLoader(data, batch_size=32, shuffle=False, num_workers=0)
src_vocab_size = data.get_src_lang_size()
tgt_vocab_size = data.get_tgt_lang_size()

d_model = 512
num_heads = 4
d_ff = 2014
max_seq_length = 100
num_layers = 6
dropout = 0.1
transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

# # Generate random sample data
# src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
# tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)

loss_criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

batch_data = next(iter(train_dataloader)) # (batch_size, en/fr, seq_length)
print((batch_data[:,0,:]).shape)
print((batch_data[:,1,:-1]).shape)
for epoch in tqdm(range(50)):
    batch_data = next(iter(train_dataloader))
    optimizer.zero_grad()
    output = transformer(batch_data[:,0,:], batch_data[:,1,:-1])

    loss = loss_criterion(output.contiguous().view(-1, output.size(2)), batch_data[:, 0, 1:].contiguous().view(-1).long())
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Reading the dataframe and storing untokenized pairs...


100%|██████████| 5000/5000 [00:00<00:00, 272523.75it/s]


Adding sentences to Langs amd geting data pairs...


100%|██████████| 5000/5000 [00:00<00:00, 7985.51it/s] 


Creating tokenized pairs of english and french sentences...


...

Epoch: 48, Loss: 1.020696759223938


 98%|█████████▊| 49/50 [01:02<00:01,  1.26s/it]

Epoch: 49, Loss: 1.0109819173812866


100%|██████████| 50/50 [01:03<00:00,  1.27s/it]

Epoch: 50, Loss: 0.9978189468383789


In [64]:
src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

# Generate random sample data
src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)

print(src_data.shape)
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in tqdm(range(1)):
    optimizer.zero_grad()
    print("src_data shape: ", src_data.shape)
    print("tgt_data shape: ", tgt_data[:, :-1].shape)
          
    output = transformer(src_data, tgt_data[:, :-1])
    print("Output Shape: ", output.shape)
    print(output.contiguous().view(-1, tgt_vocab_size).dtype)
    print(tgt_data[:, 1:].contiguous().view(-1).dtype)
    
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

torch.Size([64, 100])


  0%|          | 0/1 [00:00<?, ?it/s]

src_data shape:  torch.Size([64, 100])
tgt_data shape:  torch.Size([64, 99])
Output Shape:  torch.Size([64, 99, 5000])
torch.float32
torch.int64


100%|██████████| 1/1 [00:03<00:00,  3.75s/it]

Epoch: 1, Loss: 8.686461448669434


In [49]:
32*99

3168